In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib
from skimage import io, filters, morphology, measure, feature, future, segmentation, util
from scipy import ndimage
from skimage.color import label2rgb
import os
import numpy as np
import pandas as pd
from scipy import ndimage

## First importing the truss image

In [26]:
%matplotlib qt

In [49]:
in_folder = 'wireframe-tubes'
in_file = 'tube dimer05.tif'
truss_image = io.imread(os.path.join(in_folder, in_file))
plt.imshow(truss_image)

## Now applying Otsu thresholding to create a sure Bg and a sure Fg image

In [102]:
threshold = filters.threshold_otsu(filters.median(truss_image, selem=np.ones(shape=(15, 15))))

binary = (truss_image > threshold) # using '<' because the trusses have a lower intensity value
morphed_binary = morphology.remove_small_holes(morphology.remove_small_objects(~binary, min_size=200), area_threshold=50)

clear = segmentation.clear_border(morphed_binary)
sure_bg = morphology.binary_dilation(clear, np.ones(shape=(5, 5)))
sure_fg = morphology.binary_erosion(clear, np.ones(shape=(3, 3)))
bg_fg = np.logical_and(sure_bg, ~sure_fg)
plt.imshow(bg_fg)

## now skeletonizing individual trusses to create watershed sources, and performing watershed

In [105]:
edges = filters.frangi(~sure_fg)
eroded_edges = morphology.skeletonize(morphology.binary_erosion(edges, np.ones(shape=(12, 12))))
markers = measure.label(eroded_edges)
labels = segmentation.watershed(bg_fg, markers)
# plt.imshow(segmentation.mark_boundaries(truss_image, labels))

In [117]:
plt.imshow(label2rgb(labels, image=truss_image))

<ipython-input-117-5fd590966e7f>:1: FutureWarning: The new recommended value for bg_label is 0. Until version 0.19, the default bg_label value is -1. From version 0.19, the bg_label default value will be 0. To avoid this warning, please explicitly set bg_label value.
  plt.imshow(label2rgb(labels, image=truss_image))


In [106]:
regions = measure.regionprops(labels, intensity_image=truss_image)

region_means = [r.mean_intensity for r in regions]
plt.hist(region_means, bins=20)

from sklearn.cluster import KMeans
model = KMeans(n_clusters=2)

region_means = np.array(region_means).reshape(-1, 1)
model.fit(region_means)
bg_fg_labels = model.predict(region_means)
bg_fg_labels

classified_labels = labels.copy()
for bg_fg, region in zip(bg_fg_labels, regions):
    classified_labels[tuple(region.coords.T)] = bg_fg

plt.imshow(label2rgb(classified_labels, image=truss_image))